Link to paper: [Zenke, Ganguli - 2018](https://direct.mit.edu/neco/article/30/6/1514-1541/8378)

Zenke's [Tutorial](https://github.com/fzenke/spytorch) on Surrogate Gradient Descent using PyTorch.

To Implement:
1. LIF Neurons (maybe a class of such neurons)
2. Fast Sigmoid Function

Question:
1. How to implement spiking neural network in pytorch? 
  * use RNNs as Zenke suggests in his tutorial?

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dtype = torch.float
device = torch.device("cpu")

# Uncomment the line below to run on GPU
# device = torch.device("cuda:0") 

### Network Architecture

3 layer feed-forward neural network

In [ ]:
nb_inputs  = 100
nb_hidden  = 4
nb_outputs = 2

In [ ]:
batch_size = 256

### Spiking Neuron Model Setup

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

Since we are technically stimulating an RNN, the neurons have to be simulated for a certain number of timesteps

In [ ]:
time_step = 1e-3
nb_steps  = 200

####Weight matrices 

Initializing weights from a normal distribution, the variance is scaled with the inverse square root of the number of input connections.

Dale's Law is ignored here.

In [ ]:
weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

print("init done")

In [ ]:
#@title The Spiking Non-linearity
def spike_fn(x):
  out = torch.zeros_like(x)
  out[x > 0] = 1.0
  return out

Initialize the synaptic currents and the membrane potentials at zero. Then implement a loop that stimulates the neuron models over time, and record the membrane potential and output spikes of all trials and all neurons.

In [ ]:
# tensors initialized with zeros for synaptic current and membrane potential
syn = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
mem = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)

# two lists to record the membrane potentials and output spikes
mem_rec = []
spk_rec = []

# The simulation loop
for t in range(nb_steps):
  m_thr = mem - 1.0
  out = spike_fn(m_thr)
  rst = out.detach() # we do not want to backprop through the reset

  new_syn = alpha*syn + h1[:, t]


### LIF Neuron


In [ ]:
class LIFneuron():
  def.__init__(self, t_mem, t_syn):
    


### SuperSpike Network Implementation

In [ ]:
class SuperSpike(nn.Module):
  def.__init__(self):
    super(SuperSpike, self).__init__()
    self.layers = nn.Sequential()

  def forward(self, x):
    i

